In [1]:
#Libraries

import geopy as geo
from geopy.geocoders import Nominatim
from geopy import distance
import random
from amplpy import AMPL
import pandas as pd

In [2]:
#We are looking to find the TSP between this cities 

geolocator = Nominatim(user_agent="TSP")

n = 15
cities = ["barcelona","paris","tokyo","new york","delhi","singapore",
          "adelaide","buenos aires","dakar","helsinki","kabul","lima","miami"
          ,"vancouver","guayaquil"]

locations = [geolocator.geocode(city) for city in cities]    

#generate matrix of distances between cities (cost of each edge on our complete graph)    
dist = [[distance.distance((i.latitude,i.longitude),(j.latitude,j.longitude)).km for i in locations] for j in locations]


In [3]:
#path class containing all the necessary 
class Solution:
    path = []
    def __init__(self,path):
        self.path = path
        
    def tsp_cost(self):
        cost = 0
        for i in range(0,n):
            cost = cost + dist[self.path[i-1]][self.path[i]] 
        return cost
    def two_swap(self): 
        copy = self.path.copy()
        i, j = random.sample(range(0,n), 2)
        copy[i],copy[j] = copy[j],copy[i]
        return Solution(copy)
    def three_cycle(self):
        i, j, k = random.sample(range(0,n),3)
        copy = self.path.copy()
        copy[i],copy[j],copy[k] = copy[j],copy[k],copy[i]
        return Solution(copy)
    def twotwo_swap(self):
        copy = self.path.copy()
        i, j ,k ,l= random.sample(range(0,n), 4)
        copy[i],copy[j] = copy[j],copy[i]
        copy[k],copy[l] = copy[l],copy[k]
        return Solution(copy)

In [4]:

#Nearest neighbour implementation

#initialization
population = []

for i in range(0,n):
    initial_node = i
    path = [initial_node]
    not_visited = [j for j in range(0,n)]
    not_visited.remove(initial_node)
    #we select the closest node to the latest that we have visited until there are no more nodes
    actual = initial_node
    while len(path) < len(cities):
        potential = [dist[actual][index] for index in not_visited]
        actual = dist[actual].index(min(potential))
        path.append(actual)
        not_visited.remove(actual)
    population.append(Solution(path))
score = [route.tsp_cost() for route in population]
print(score)

[77929.31389626306, 78583.35937416987, 68733.42050013223, 75123.00722560791, 65889.16324545049, 62989.27133455657, 62989.271334556586, 65994.31925152114, 70407.61853179106, 75948.93605057814, 69402.40705793686, 75093.87495088603, 68918.2185306606, 62186.891507490946, 75896.2535642983]


In [5]:
#improving our initial solution using a genetic algorithm

#INITIAL POPULATION (10 1-SWAP 10 3-CYCLE FROM OG and pick the best 10 ones) 
initial_population_2swap = [path.two_swap() for k in range(10) for path in population]
initial_population_3cycle = [path.three_cycle() for l in range(10) for path in population]
n_generations = 10
population = population + initial_population_2swap +  initial_population_3cycle
score = [route.tsp_cost() for route in population]

#variables to keep track of the best found solution
best_score, best_path_idx = min((ite,value) for (value,ite) in enumerate(score))
best_path = population[best_path_idx]


for i in range(0,n_generations):
    #create new generation
    print(i)
    print(best_score)
    population_2swap = [path.two_swap() for k in range(5) for path in population]
    population_3cycle = [path.three_cycle() for k in range(5) for path in population]
    population_22swap = [path.twotwo_swap() for k in range(5) for path in population]
    new_generation = population + population_2swap + population_3cycle + population_22swap
    
    #check if a better solution has been found
    score = [path.tsp_cost() for path in new_generation]
    if min(score) < best_score:
        best_score, best_path_idx = min((ite,value) for (value,ite) in enumerate(score))
        best_path = new_generation[best_path_idx]
        
    #keep best elements   
    population = []
    for j in range(0,50):
        candidate, candidate_path_idx = min((ite,value) for (value,ite) in enumerate(score))
        population.append(new_generation[candidate_path_idx])
        score.remove(candidate)
        
    
print(best_path)
print(best_score)
print([cities[i] for i in best_path])

0
62186.891507490946
1
62186.891507490946
2
62186.891507490946
3
62186.891507490946
4
62186.891507490946
5
62186.891507490946
6
62186.891507490946
7
62186.891507490946
8
62186.891507490946
9
62186.891507490946
62186.891507490946


TypeError: 'Solution' object is not iterable

In [8]:
##Upper bound lr ampl use

def ampl_solve_lr(n,dist,model):
    #generates a "Kcluster.dat" with whe data for the LP problem for AMPL to solve
    output = open("TSP.dat", "w")
    output.write('param n:= '+ str(n) + ';')
    output.write('\n')
    output.write('param dist: ')
    for i in range(n):output.write(str(i + 1) + ' ')
    output.write(':=')
    output.write('\n')
    for i in range(n):
        output.write(str(i + 1) + ' ')
        for j in range(n):
            output.write(str(dist[i][j]))
            if(j != n-1):output.write(' ')
        if(i != n-1):output.write('\n')
    output.write(';')
    output.write('\n')
    output.close()
    ampl = AMPL()
    ampl.read(model)
    ampl.read_data("TSP.dat")
    ampl.option["solver"] = "highs"
    ampl.solve()
    solution_input = ampl.get_variable("edge").get_values().to_pandas().values.tolist()
    sol = [[0 for _ in range(n)] for _ in range(n)]
    for i in range(n): 
        for j in range(n): 
            sol[i][j] = solution_input[i*n + j][0]      
    return sol

In [18]:
#SEC identification heuristic method
def SEC_identification_heuristic(graph,n):
    ite = [i for i in range(n)]
    sec_elements = [[i] for i in range(n)]
    while(True):
        
        for i in ite:
            for j in ite:
                if graph[i][j] > 1:
                    if (len(sec_elements[i]) + len(sec_elements[j]) < n/2): return("A SEC has been found compromising nodes" + str(sec_elements[i]) +" and " + str(sec_elements[j]))
                    return("No SEC has been found by the algorithm")
                
                
        ite_finished = False
        for i in ite:
            for j in ite: 
                if graph[i][j] == 1 and not ite_finished:
                    ite.remove(j)
                    sec_elements[i].append(j)
                    for k in ite: 
                        if(k != i and k != j): 
                            graph[i][k],graph[k][i] = graph[i][k] + graph[j][k],graph[k][i] + graph[k][j]
                            ite_finished = True
                        
                        

In [19]:
#solving lr
lr_sol = ampl_solve_lr(n,dist,"TSP_relaxed.mod")
SEC_identification_heuristic(lr_sol.copy(),n)



HiGHS 1.6.0:HiGHS 1.6.0: optimal solution; objective 55198.62519
18 simplex iterations
0 barrier iterations
 


'A SEC has been found compromising nodes[0, 8] and [1]'

In [20]:
lr_sec1 = ampl_solve_lr(n,dist,"TSP_sec1.mod")
SEC_identification_heuristic(lr_sec1.copy(),n)


HiGHS 1.6.0:HiGHS 1.6.0: optimal solution; objective 55556.56531
25 simplex iterations
0 barrier iterations
 


'A SEC has been found compromising nodes[0, 8, 1] and [9]'

In [21]:

lr_sec2 = ampl_solve_lr(n,dist,"TSP_sec2.mod")
SEC_identification_heuristic(lr_sec2.copy(),n)

HiGHS 1.6.0:HiGHS 1.6.0: optimal solution; objective 56002.99088
26 simplex iterations
0 barrier iterations
 


'A SEC has been found compromising nodes[2, 6] and [5]'

In [22]:
lr_sec3 = ampl_solve_lr(n,dist,"TSP_sec3.mod")
SEC_identification_heuristic(lr_sec3.copy(),n)

HiGHS 1.6.0:HiGHS 1.6.0: optimal solution; objective 56442.74234
22 simplex iterations
0 barrier iterations
 


'No SEC has been found by the algorithm'